In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from img_utils import automatic_brightness_and_contrast, find_if_close


In this project, you are to apply image segmentation techniques to X-ray angiography, where X-ray images are taken when an X-ray absorbing substance is injected into the patient's blood stream to produce contrast. The resulting X-ray images have dark regions representing the blood flow within vessels. Your system should be able to automatically locate any occlusion and follow the surrounding vessel wall to compute the ratio between the minimum and nominal vessel diameters. Such results are practically important in detecting coronary disease. Your system should also accept user input of occlusion locations and perform the same percent occlusion measurement in that particular area.

In [2]:
images_dir = '../sample-images'
image_names = os.listdir(images_dir)
# image_name = shuffle(image_names)

In [3]:
%matplotlib tk
fig, ax = plt.subplots(2, 1, figsize = (20,10))

for idx, image_name in enumerate(image_names):
    
    full_path = f'{images_dir}/{image_name}' 
    img = cv2.imread(full_path)
    x_new = int(img.shape[1] * .1)
    y_new = int(img.shape[0] * .1)
    img = img[y_new:img.shape[0] - y_new, x_new: img.shape[1] - x_new]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = automatic_brightness_and_contrast(gray)
#     blurred = cv2.GaussianBlur(gray, (9,9), 0)
    blurred = cv2.medianBlur(gray, 3)
#     blurred = automatic_brightness_and_contrast(blurred)
    block_size = int(img.shape[0] / 12 )
    block_size = block_size + 1 if block_size % 2 == 0 else block_size
    
    edged_adap = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, 10)
#     edged_adap = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU, block_size)[0]

    min_contour_area = 1000
    
    thresh = cv2.morphologyEx(edged_adap, cv2.MORPH_ELLIPSE, np.ones((3,3), np.uint8))
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contours =[i for i in contours if cv2.contourArea(i) > min_contour_area]
    edged_contour = img.copy()
    new_contours = []
    for idx1, c1 in enumerate(contours):
        cv2.drawContours(edged_contour, [c1], -1, (0,0,0), -1)
        if idx1 == len(contours) - 1:
            break
#         for idx2, c2 in enumerate(contours):
#             dist = find_if_close(c1, c2, 5)
#             if dist:
#                 hulled = cv2.convexHull(np.vstack([c1, c2]))
#                 new_contours.append(hulled)
                
#     contour_test = img.copy()
#     cv2.drawContours(contour_test, new_contours, -1, (255,0,0), -1)
#     contour_test = cv2.cvtColor(contour_test, cv2.COLOR_BGR2GRAY)
#     contour_test = cv2.threshold(contour_test, 0, 255, cv2.THRESH_BINARY_INV)[1]
    edged_contour = cv2.cvtColor(edged_contour, cv2.COLOR_BGR2GRAY)
    threshed = cv2.threshold(edged_contour, 0, 255, cv2.THRESH_BINARY_INV)[1]
#     edged_adap_2 = cv2.adaptiveThreshold(edged_contour, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, 10)
#     edge_mask = cv2.threshold(edged_contour, 254, 255, cv2.THRESH_BINARY_INV)[0]
    dilated = cv2.dilate(threshed, None, iterations = 15)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (7,7))
    closing = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
    
    mask = cv2.bitwise_xor(threshed.copy(), dilated)
#     mask = cv2.bitwise_not(blurred.copy(), blurred.copy(), threshed)
    stacked = np.hstack([blurred, edged_adap, edged_contour, threshed, dilated, mask])
    ax[idx].imshow(stacked, cmap = 'gray')
    ax[idx].set_title(image_name)
    
#     ax[1].imshow(contour_test)
#     break

# plt.show()

    

    

# Testing Block Sizes

In [4]:
%matplotlib tk

for image_name in image_names:
    
    full_path = f'{images_dir}/{image_name}' 
    img = cv2.imread(full_path)
    x_new = int(img.shape[1] * .1)
    y_new = int(img.shape[0] * .1)
    img = img[y_new:img.shape[0] - y_new, x_new: img.shape[1] - x_new]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = automatic_brightness_and_contrast(gray)
#     blurred = cv2.GaussianBlur(gray, (7,7), 0)
    blurred = cv2.medianBlur(gray, 15)
#     blurred = cv2.blur(gray, (9,9))
#     blurred = cv2.bilateralFilter(gray, 15, 350, 350)
    blurred = automatic_brightness_and_contrast(blurred)
    stacked = []
    blocks = list(range(6, 23, 2))
    block_values = []
    for i in blocks:
        block_size = int(img.shape[0] / i)
        block_size = block_size + 1 if block_size % 2 == 0 else block_size
        block_values.append(block_size)
        edged = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size, 10)
        stacked.append(edged)
    fig, ax = plt.subplots(1, len(blocks) + 1, figsize = (10,10))
    ax[0].imshow(blurred, cmap = 'gray')
    ax[0].set_title('Original Image')
    for idx, (edge_img, block, block_value) in enumerate(zip(stacked, blocks, block_values)):
        idx += 1
        ax[idx].imshow(edge_img, cmap = 'gray')
        ax[idx].set_title(f'Block Size {block_value} ({block})')
    plt.tight_layout()
    plt.show()
        
#     stacked = np.hstack(stacked)
#     plt.imshow(stacked, cmap = 'gray')
#     plt.show()

    